In [6]:
import pandas
import re

data = pandas.read_excel("AMAM eMuseum data export 9.26.24.xlsx")
data

,ObjectID,Accession Number,Object Number,Sort Number,Department,Classification,Acquisition Method,Object Status,Artist/Maker,Title,Object Name (Ctrl + Insert),Dated,Materials / Techniques,Dimensions,Description,Credit Line (Ctrl + Insert),Culture (Ctrl + Insert),Period/Dynasty (Ctrl + Insert),eMuseum Label Text
0,10739,1894.1,NaN,1894 1,Asian Art,Ceramics,Gift,Permanent Collection,Chinese,Crackle-ware Vase,vase,19th century,Glazed porcelain,Overall: 8 1/2 × 5 in. (21.6 × 12.7 cm),Globular form vase with tall straight neck; gr...,Gift of D. A. Bunker (OC 1883),Chinese,Qing Dynasty (1644-1911),Crackle-ware may have begun as a mistake but i...
1,4249,1900.4,NaN,1900 4,Asian Art,Prints,Found in Collection,Permanent Collection,Utagawa Hiroshige I 初代目歌川広重,The Pine Tree of the Feathered Robe at Miho Ba...,woodblock print,1856,Color woodblock print,Image/Sheet: 2 × 13 3/16 in. (5.1 × 33.5 cm),Tokaido Road\n,Allen Memorial Art Museum,Japanese,Edo period (1615–1868),"This small woodblock print, with a white image..."
2,2727,1900.28,NaN,1900 28,European Art,Prints,Found in Collection,Permanent Collection,Jean-François Millet,"Peasant Returning from the Manure Heap, or Man...",etching,1855–56,Etching,Image: 6 1/4 × 5 in. (15.9 × 12.7 cm)\nPlate: ...,NaN,Allen Memorial Art Museum,French,NaN,Millet focused on the human figure – rather th...
3,12652,1900.114,NaN,1900 114,American Art,Prints,Gift,Permanent Collection,James S. Smillie,Voyage of Life: Youth,engraving,ca. 1853,Engraving,Image: 15 × 22 1/4 in. (38.1 × 56.5 cm)\nSheet...,One of a series of four engravings after paint...,Gift of Genevieve Brandt (OC 1904),American,NaN,Cole's Voyage of Life (1839-40) depicts life's...
4,2548,1900.123C,NaN,1900 123C,Pre-Columbian and American Indian,Sculpture,Found in Collection,Permanent Collection,Tesuque Pueblo,Statuette Representing a Rain God,statuette,early 20th century,Pottery,Overall: 5 13/16 × 2 1/4 × 3 3/4 in. (14.8 × 5...,One of three pinkish-gray pottery statuettes o...,Allen Memorial Art Museum,Tesuque Pueblo,NaN,This statuette was made as an imitation of a r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2144,40978,2023.59,NaN,2023 59,Modern and Contemporary Art,Sculpture,Purchase,Permanent Collection,Sahar Khoury,Untitled (red with belts),sculpture,2022,"Glazed ceramic, powder-coated steel, and belts",Overall: 21 × 12 × 12 in. (53.3 × 30.5 × 30.5 cm),NaN,Purchased with funds from Carl Read Gerber (OC...,American,NaN,"Sahar Khoury (b. 1973 in Chicago, IL) lives an..."
2145,40987,2024.2,NaN,2024 2,European Art,Paintings,Purchase,Permanent Collection,Lady Laura Theresa Alma-Tadema,The Bible Lesson,painting,ca. 1883,Oil on canvas,Overall: 25 5/8 × 20 in. (65.1 × 50.8 cm)\nFra...,A young girl in a silk dress describes bible s...,Art Object Sales Fund,British,NaN,Although this work was painted in England towa...
2146,41009,2024.13,NaN,2024 13,Modern and Contemporary Art,Drawings / Watercolors,Purchase,Permanent Collection,Loïs Mailou Jones,Symbols d'Afrique,drawing,1983,"Acrylic, ink, and pencil on paper",Image: 12 1/2 × 8 in. (31.8 × 20.3 cm)\nSheet:...,NaN,Ruth C. Roush Contemporary Art Fund,American,NaN,Loïs Mailou Jones was born and raised in Bosto...
2147,41049,2024.21,NaN,2024 21,Modern and Contemporary Art,Paintings,Purchase,Permanent Collection,Jay Lynn Gomez,self portrait as a trans woman soccer logo,painting,2024,Acrylic on cardboard medication packaging,Image: 4 1/2 × 6 1/4 in. (11.4 × 15.9 cm)\nFra...,NaN,Purchased with funds from Carl Read Gerber (OC...,American,NaN,<i>Self portrait as a trans woman soccer logo<...


In [3]:
def hyphenate(entry):
    entry = entry.replace("—", "-")
    entry = entry.replace("–", "-")
    return entry


def parse(items):
    dates = pandas.DataFrame(columns=["ObjectID", "Start Year", "End Year", "Approximate"])
    for row in items.itertuples():
        id = row.ObjectID
        entry = row.Dated
        m = re.fullmatch(r"(\d+)(?:st|nd|rd|th) century(?: CE)?", entry)
        if m:
            century = int(m.group(1)) * 100
            dates.loc[len(dates)] = [id, century-99, century, False]
            continue
        m = re.fullmatch(r"(?:\w+\s+)?(\d+)", entry)
        if m:
            year = int(m.group(1))
            dates.loc[len(dates)] = [id, year, year, False]
            continue
        m = re.fullmatch(r"(?:printed )?(\d{2,})-(\d{2})(?: \(restrike\))?", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year - (start_year % 100) + int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"(?:ca\. |circa |ca\.|after )(\d+)", entry)
        if m:
            year = int(m.group(1))
            dates.loc[len(dates)] = [id, year, year, True]
            continue
        m = re.fullmatch(r"(?:Probably |)early (\d{1,2})(?:st|nd|rd|th) century(?: CE)?", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 99
            end_year = start_year + 33
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)-(\d{1,2})(?:st|nd|rd|th) (?:C|c)entur(?:y|ies)", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 99
            end_year = start_year + 99
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"late (\d{1,2})(?:st|nd|rd|th) century(?: CE|)", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 33
            end_year = start_year + 33
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d{3}0)s", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year + 9
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"second half (\d{1,2})(?:st|nd|rd|th)-early (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 50
            end_year = (int(m.group(2)) * 100) - 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"late (\d{1,2})(?:st|nd|rd|th)\s*(?:-|to )early (\d{1,2})(?:st|nd|rd|th)\s*century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 33
            end_year = (int(m.group(2)) * 100) - 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(?:ca\. |circa )(\d{2,})-(\d{2})", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year - start_year % 100 + int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"mid-(\d{1,2})(?:st|nd|rd|th)(?:-| to )early (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 66
            end_year = (int(m.group(2)) * 100) - 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d{3,})-(\d{3,})", entry)
        if m:
            start_year = int(m.group(1))
            end_year = int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"(\d+) cast of (?:ca.|circa) (\d+) original", entry)
        if m:
            year1 = int(m.group(1))
            year2 = int(m.group(2))
            dates.loc[len(dates)] = [id, year1, year1, False]
            dates.loc[len(dates)] = [id, year2, year2, True]
            continue
        m = re.fullmatch(r"early (\d{1,2})(?:st|nd|rd|th)-early (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 99
            end_year = int(m.group(2)) * 100 - 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"late (\d{1,2})(?:st|nd|rd|th)-(\d{1,2})(?:st|nd|rd|th) +century", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 33
            end_year = int(m.group(2)) * 100
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"mid-(\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 66
            end_year = start_year + 33
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d+), (?:published|reworked in|repainted in|printed(?: in)?) (\d+)", entry)
        if m:
            year1 = int(m.group(1))
            year2 = int(m.group(2))
            dates.loc[len(dates)] = [id, year1, year1, False]
            dates.loc[len(dates)] = [id, year2, year2, False]
            continue
        m = re.fullmatch(r"mid-(\d{1,2})(?:st|nd|rd|th) century CE \(after a (\d{1,2})(?:st|nd|rd|th) century BCE Greek original\)", entry)
        if m:
            start_year1 = int(m.group(1)) * 100 - 66
            start_year2 = start_year1 + 33
            end_year1 = -(int(m.group(2)) * 100)
            end_year2 = end_year1 + 99
            dates.loc[len(dates)] = [id, start_year1, start_year2, True]
            dates.loc[len(dates)] = [id, end_year1, end_year2, False]
            continue
        m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)-early (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 99
            end_year = int(m.group(2)) * 100 - 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)-(\d{1,2})(?:st|nd|rd|th) century CE \(after a (\d{1,2})(?:st|nd|rd|th) century BCE Greek original\)", entry)
        if m:
            start_year1 = int(m.group(1)) * 100 - 99
            start_year2 = int(m.group(2)) * 100
            end_year1 = -(int(m.group(3)) * 100)
            end_year2 = end_year1 + 99
            dates.loc[len(dates)] = [id, start_year1, start_year2, False]
            dates.loc[len(dates)] = [id, end_year1, end_year2, False]
            continue
        m = re.fullmatch(r"ca\. (\d+)-(\d+) BC(?:E|)", entry)
        if m:
            start_year = -int(m.group(1))
            end_year = -int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"late (\d{2,3}0)s", entry)
        if m:
            start_year = int(m.group(1)) + 6
            end_year = start_year + 3
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d+)-(\d+) BC(?:E|)", entry)
        if m:
            start_year = -int(m.group(1))
            end_year = -int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"ca\. (\d+) BC(?:E|)", entry)
        if m:
            start_year = -int(m.group(1))
            dates.loc[len(dates)] = [id, start_year, start_year, True]
            continue
        m = re.fullmatch(r"(\d{1,})-(\d{1})", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year - (start_year % 10) + int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)-(\d{1,2})(?:st|nd|rd|th) century \(imitation of (\d+) original\)", entry)
        if m:
            start_year1 = int(m.group(1)) * 100 - 99
            end_year1 = int(m.group(2)) * 100
            start_year2 = int(m.group(3))
            dates.loc[len(dates)] = [id, start_year1, end_year1, False]
            dates.loc[len(dates)] = [id, start_year2, start_year2, False]
            continue
        m = re.fullmatch(r"(?:ca\. |circa )(\d{1,2})(?:st|nd|rd|th) century BCE", entry)
        if m:
            start_year = -int(m.group(1)) * 100
            end_year = start_year + 99
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)-(\d{1,2})(?:st|nd|rd|th) (?:C|c)entury BC(?:E|)", entry)
        if m:
            start_year = -int(m.group(1)) * 100
            end_year = -int(m.group(2)) * 100 - 99
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"second half (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 50
            end_year = start_year + 50
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"first half (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 99
            end_year = start_year + 49
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"early to mid-(\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 99
            end_year = start_year + 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"late (\d{1,3}0)s-early (\d{1,3}0)s", entry)
        if m:
            start_year = int(m.group(1)) + 6
            end_year = int(m.group(2)) + 3
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"mid- to late (\d{1,3}0)s", entry)
        if m:
            start_year = int(m.group(1)) + 3
            end_year = start_year + 6
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"early (\d{1,3}0)s", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year + 3
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"mid-(\d{1,3}0)s", entry)
        if m:
            start_year = int(m.group(1)) + 3
            end_year = start_year + 3
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"early (\d{1,2})(?:st|nd|rd|th) century, original (\d{2,})-(\d{2})", entry)
        if m:
            end_year1 = int(m.group(1)) * 100
            start_year1 = end_year1 - 99
            start_year2 = int(m.group(2))
            end_year2 = start_year2 - (start_year2 % 100) + int(m.group(3))
            dates.loc[len(dates)] = [id, start_year1, end_year1, True]
            dates.loc[len(dates)] = [id, start_year2, end_year2, False]
            continue
        m = re.fullmatch(r"(\d{1,3}0)s-(\d0)s", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year - (start_year % 100) + int(m.group(2)) + 9
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"ca\. (\d{2,})-(\d{2}).+,.+(\d{2,})-(\d{2})", entry)
        if m:
            start_year1 = int(m.group(1))
            end_year1 = start_year1 - (start_year1 % 100) + int(m.group(2))
            start_year2 = int(m.group(3))
            end_year2 = start_year2 - (start_year2 % 100) + int(m.group(4))
            dates.loc[len(dates)] = [id, start_year1, end_year1, True]
            dates.loc[len(dates)] = [id, start_year2, end_year2, False]
            continue
        m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)(?:-| to )mid-(\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 99
            end_year = (int(m.group(2)) * 100) - 33
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(?:ca\. |circa |ca\.)(\d+)-(\d+)", entry)
        if m:
            start_year = int(m.group(1))
            end_year = int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        if entry == "20th century pastiche using components from second quarter 16th century":
            dates.loc[len(dates)] = [id, 1901, 2000, False]
            dates.loc[len(dates)] = [id, 1525, 1550, False]
            continue
        m = re.fullmatch(r"(\d+), (?:published|reworked in|printed) (?:ca\. |circa )(\d+)", entry)
        if m:
            year1 = int(m.group(1))
            year2 = int(m.group(2))
            dates.loc[len(dates)] = [id, year1, year1, False]
            dates.loc[len(dates)] = [id, year2, year2, True]
            continue
        m = re.fullmatch(r"\w* ?\w* \d{1,2}, (\d+)", entry)
        if m:
            year1 = int(m.group(1))
            dates.loc[len(dates)] = [id, year1, year1, False]
            continue
        m = re.fullmatch(r"(\d{2,})-(\d{2}), (?:published|reworked in|printed) (\d+)", entry)
        if m:
            year1 = int(m.group(1))
            year2 = year1 - (year1 % 100) + int(m.group(2))
            year3 = int(m.group(3))
            dates.loc[len(dates)] = [id, year1, year2, False]
            dates.loc[len(dates)] = [id, year3, year3, False]
            continue
        if entry == "1970 (enlarged refabrication of 1964 project)":
            dates.loc[len(dates)] = [id, 1964, 1964, False]
            dates.loc[len(dates)] = [id, 1970, 1970, False]
            continue
        m = re.fullmatch(r"(\d+)-(\d+), (?:published|reworked in|printed) (\d+)", entry)
        if m:
            year1 = int(m.group(1))
            year2 = int(m.group(2))
            year3 = int(m.group(3))
            dates.loc[len(dates)] = [id, year1, year2, False]
            dates.loc[len(dates)] = [id, year3, year3, False]
            continue
        if entry == "ca. 1970 (modern print from 1896 negative)":
            dates.loc[len(dates)] = [id, 1970, 1970, True]
            dates.loc[len(dates)] = [id, 1896, 1896, False]
            continue
        m = re.fullmatch(r"(\d+), (?:published|reworked in|printed) (\d{2,})-(\d{2})", entry)
        if m:
            year1 = int(m.group(1))
            year2 = int(m.group(2))
            year3 = year2 - (year2 % 100) + int(m.group(3))
            dates.loc[len(dates)] = [id, year1, year1, False]
            dates.loc[len(dates)] = [id, year2, year3, False]
            continue
        if entry == "negative ca. 1865, printed early 1870s":
            dates.loc[len(dates)] = [id, 1865, 1865, True]
            dates.loc[len(dates)] = [id, 1870, 1873, True]
            continue
        m = re.fullmatch(r"[Ee]arly (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 99
            end_year = start_year + 33
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(?:ca\. |circa )(\d{3}0)s", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year + 9
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        if entry == "n.d.":
            dates.loc[len(dates)] = [id, 0, 0, True]
            continue
        if entry == "1931, printed later":
            dates.loc[len(dates)] = [id, 1931, 1931, False]
            continue
        m = re.fullmatch(r"(?:ca\. |circa )(\d{2,})-(\d{2}), (?:published|reworked in|printed) (\d+)", entry)
        if m:
            year1 = int(m.group(1))
            year2 = year1 - (year1 % 100) + int(m.group(2))
            year3 = int(m.group(3))
            dates.loc[len(dates)] = [id, year1, year2, True]
            dates.loc[len(dates)] = [id, year3, year3, False]
            continue
        m = re.fullmatch(r"mid-? to late (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 66
            end_year = start_year + 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        if entry == "late 19th-early 20th century (heCE replaced 1930-59)":
            dates.loc[len(dates)] = [id, 1867, 1933, True]
            dates.loc[len(dates)] = [id, 1930, 1959, False]
            continue
        if entry == "2009- (letter transcribed in 2018)":
            dates.loc[len(dates)] = [id, 2009, 2009, False]
            dates.loc[len(dates)] = [id, 2018, 2018, False]
            continue
        m = re.fullmatch(r"(\d+)(?:st|nd|rd|th) century BCE", entry)
        if m:
            century = -(int(m.group(1)) * 100)
            dates.loc[len(dates)] = [id, century, century + 99, False]
            continue
        m = re.fullmatch(r"(\d+)-(\d+),? \((?:published|reworked in|printed) (\d+)\)", entry)
        if m:
            year1 = int(m.group(1))
            year2 = int(m.group(2))
            year3 = int(m.group(3))
            dates.loc[len(dates)] = [id, year1, year2, False]
            dates.loc[len(dates)] = [id, year3, year3, False]
            continue

        print(dates)
        raise ValueError(f"Failed to parse {entry}")
    return dates

data["Dated"] = data["Dated"].map(hyphenate)
dates_pre = data[["ObjectID", "Dated"]]
dates = parse(dates_pre)
dates

,ObjectID,Start Year,End Year,Approximate
0,10739,1801,1900,False
1,4249,1856,1856,False
2,2727,1855,1856,False
3,12652,1853,1853,True
4,2548,1901,1934,True
...,...,...,...,...
2226,40978,2022,2022,False
2227,40987,1883,1883,True
2228,41009,1983,1983,False
2229,41049,2024,2024,False


In [36]:
FOOT_CM = 30.48
IN_CM = 2.54


def dimensions(items):
    dims = pandas.DataFrame(columns=["ObjectID", "Type", "Axis", "Value (cm/min/kg)"])
    for row in items.itertuples():
        id = row.ObjectID
        entry = row.Dimensions
        listings = entry.splitlines()
        for listing in listings:
            if listing == "Overall: 6 1/4 × 3 1/8 × 8 7/16 × 1 1/4 in. (15.9 × 7.9 × 21.4 × 3.2 cm)":
                dims.loc[len(dims)] = [id, "Overall", "Length", 15.9]
                dims.loc[len(dims)] = [id, "Overall", "Length", 7.9]
                dims.loc[len(dims)] = [id, "Overall", "Length", 21.4]
                dims.loc[len(dims)] = [id, "Overall", "Length", 3.2]
                continue
            m = re.match(r"(.+):", listing)
            if not m:
                print(f"No dimensions found for {listing}")
                continue
            type = m.group(1)
            is_width = bool(re.match(r"[Ww]idth", type))
            is_length = bool(re.match(r"[Ll]ength", type))
            is_depth = bool(re.match(r"[Dd]epth", type))
            if is_width + is_length + is_depth > 1:
                raise ValueError(f"Failed to parse {listing}")
            if type == "Duration":
                n = re.search(r"(\d+\.?\d*)", listing)
                if not n:
                    raise ValueError(f"Failed to parse {listing}")
                duration = float(n.group(1))
                dims.loc[len(dims)] = [id, type, "Duration", duration]
                continue
            if type.find("Weight") != -1:
                kg_section = listing.split("(")[-1]
                n = re.match(r"(\d+\.?\d*)", kg_section)
                if not n:
                    raise ValueError(f"Failed to parse {listing}")
                weight = float(n.group(1))
                dims.loc[len(dims)] = [id, type, "Weight", weight]
                continue
            cm_section = listing.split("(")[-1]
            full = re.fullmatch(r"(\d+\.?\d*) × (\d+\.?\d*) × (\d+\.?\d*) cm, (\d+\.?\d*) kg\)", cm_section)
            if full:
                dims.loc[len(dims)] = [id, "Overall", "Length", float(full.group(1))]
                dims.loc[len(dims)] = [id, "Overall", "Width", float(full.group(2))]
                dims.loc[len(dims)] = [id, "Overall", "Depth", float(full.group(3))]
                dims.loc[len(dims)] = [id, "Overall", "Weight", float(full.group(4))]
                continue
            is_cm = cm_section.find("cm")
            is_in = cm_section.find("in")
            is_ft = cm_section.find("ft")
            is_m = cm_section.find(" m")
            if is_cm + is_ft + is_in + is_m == -4:
                raise ValueError(f"Failed to parse {listing}")
            n = re.findall(r"(\d+\.?\d*)", cm_section)
            if not n or len(n) > 3:
                raise ValueError(f"Failed to parse {listing}")
            n = list(map(float, n))
            if is_ft != -1:
                n = list(map(lambda x: round(x * FOOT_CM, 1), n))
            if is_in != -1:
                n = list(map(lambda x: round(x * IN_CM, 1), n))
            if is_m != -1:
                n = list(map(lambda x: round(x * 100, 1), n))
            match len(n):
                case 1:
                    if is_width:
                        dims.loc[len(dims)] = [id, type, "Width", n[0]]
                        continue
                    if is_length:
                        dims.loc[len(dims)] = [id, type, "Length", n[0]]
                        continue
                    if is_depth:
                        dims.loc[len(dims)] = [id, type, "Depth", n[0]]
                        continue
                    dims.loc[len(dims)] = [id, type, "Length", n[0]]
                    continue
                case 2:
                    dims.loc[len(dims)] = [id, type, "Length", n[0]]
                    dims.loc[len(dims)] = [id, type, "Width", n[1]]
                    continue
                case 3:
                    dims.loc[len(dims)] = [id, type, "Length", n[0]]
                    dims.loc[len(dims)] = [id, type, "Width", n[1]]
                    dims.loc[len(dims)] = [id, type, "Depth", n[2]]
                    continue
    return dims

dims = data[["ObjectID", "Dimensions"]]
dim_table = dimensions(dims)
dim_table

No dimensions found for Vertical ōban triptych


,ObjectID,Type,Axis,Value (cm/min/kg)
0,10739,Overall,Length,21.6
1,10739,Overall,Width,12.7
2,4249,Image/Sheet,Length,5.1
3,4249,Image/Sheet,Width,33.5
4,2727,Image,Length,15.9
...,...,...,...,...
7467,38881,Overall (pin),Width,1.0
7468,38881,Overall (pin),Depth,3.0
7469,38881,Overall (box),Length,5.1
7470,38881,Overall (box),Width,6.4


In [60]:
def extract_materials(mls):
    ms = []
    object_has_material = pandas.DataFrame(columns=["ObjectID", "MaterialID"])
    for row in mls.itertuples():
        id = row.ObjectID
        materials = row[2]
        m_list = re.split(r" on | and | with |,|\/| in |&|\(|\)|;", materials)
        m_list = list(map(lambda x: x.strip(), m_list))
        m_list = list(map(lambda x: x.casefold(), m_list))
        for material in m_list:
            if not material:
                continue
            if material not in ms:
                ms.append(material)
            m_id = ms.index(material)
            object_has_material.loc[len(object_has_material)] = [id, m_id]
    return ms, object_has_material

mls = data[["ObjectID", "Materials / Techniques"]]
materials, object_has_material = extract_materials(mls)
materials = pandas.DataFrame({"Material": materials})
materials["MaterialID"] = materials.index
materials

,Material,MaterialID
0,glazed porcelain,0
1,color woodblock print,1
2,etching,2
3,engraving,3
4,pottery,4
...,...,...
933,cardboard medication packaging,933
934,enamel pin,934
935,box offset-printed,935
936,black-and-white,936


In [61]:
import sqlite3

con = sqlite3.connect("museum.db")
data.to_sql("objects", con, if_exists="replace", index=False)
dates.to_sql("dates", con, if_exists="replace", index=False)
dim_table.to_sql("dimensions", con, if_exists="replace", index=False)
materials.to_sql("materials", con, if_exists="replace", index=False)
object_has_material.to_sql("object_has_material", con, if_exists="replace", index=False)

4946